## Model Train 1 - Tesis Javier-Uriel

### Importamos algunas librerías que nos serán útiles más adelante

In [1]:
import os
import json
import random
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import models, layers
assert (tf.__version__=='2.4.1'), 'Versión incorrecta de Tensorflow, por favor instale 2.4.1'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from IPython.display import clear_output

pd.set_option('display.max_columns', None) #Para mostrar todas las columnas

import gc #garbage collector

Num GPUs Available:  1


In [2]:
gpus = tf.config.list_physical_devices('GPU') 
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


### Leemos el Dataset

In [3]:
#Dataset solo movimientos en Z
dataset_name = "Dataset_Z3"
directory = "../logs/Datasets/"+dataset_name
ORDER = 3  #Número de estados retrasados
rpm_list = ['RPM0', 'RPM1', 'RPM2', 'RPM3'] #Lista de Acciones
states_list = ["vz","az", "uvz"]#, #Lista de Estados
#                    "p", "q",
#                    "wp", "wq", 
#                    "ap", "aq"]

In [4]:
def pandas_read():
    dfs = []
    global states_list
    # reading train data
    for filename in os.listdir(directory):
        if not filename.endswith(".csv"):
            continue
        df = pd.read_csv(os.path.join(directory, filename))
        a = []
        ## Desplazamos estados anteriores        
        for n in range(1,ORDER+1):
            for column in states_list:
                df[column+str(n)] = df[column].shift(periods=n, fill_value=0)
                a.append(column+str(n))
        dfs.append(df)
    states_list+=a       

    return pd.concat(dfs)

In [5]:
dataset = pandas_read()
gc.collect()

22

In [6]:
dataset.head()

,timestamps,x,y,z,Q1,Q2,Q3,Q4,p,q,r,vx,vy,vz,wp,wq,wr,ax,ay,az,ap,aq,ar,RPM0,RPM1,RPM2,RPM3,ux,uy,uz,uvx,uvy,uvz,up,uq,ur,uwp,uwq,uwr,vz1,vz2,vz3,az1,az2,az3,uvz1,uvz2,uvz3
0,0.000000,0.0,0.0,24.999830,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.040833,0.0,0.0,0.0,0.0,0.0,-9.800000,0.0,0.0,0.0,21666.4475,21666.4475,21666.4475,21666.4475,0.0,0.0,25.0,0.0,0.0,1.907893,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
1,0.004167,0.0,0.0,24.999871,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,0.009909,0.0,0.0,0.0,0.0,0.0,12.178210,0.0,0.0,0.0,21666.4475,21666.4475,21666.4475,21666.4475,0.0,0.0,25.0,0.0,0.0,1.907893,0.0,0.0,0.0,0.0,0.0,0.0,-0.040833,0.000000,0.000000,-9.800000,0.00000,0.00000,1.907893,0.000000,0.000000
2,0.008333,0.0,0.0,25.000124,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,0.060643,0.0,0.0,0.0,0.0,0.0,12.176110,0.0,0.0,0.0,21666.4475,21666.4475,21666.4475,21666.4475,0.0,0.0,25.0,0.0,0.0,1.907893,0.0,0.0,0.0,0.0,0.0,0.0,0.009909,-0.040833,0.000000,12.178210,-9.80000,0.00000,1.907893,1.907893,0.000000
3,0.012500,0.0,0.0,25.000588,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,0.111368,0.0,0.0,0.0,0.0,0.0,12.173937,0.0,0.0,0.0,21666.4475,21666.4475,21666.4475,21666.4475,0.0,0.0,25.0,0.0,0.0,1.907893,0.0,0.0,0.0,0.0,0.0,0.0,0.060643,0.009909,-0.040833,12.176110,12.17821,-9.80000,1.907893,1.907893,1.907893
4,0.016667,0.0,0.0,25.001263,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,0.162083,0.0,0.0,0.0,0.0,0.0,12.171559,0.0,0.0,0.0,21666.4475,21666.4475,21666.4475,21666.4475,0.0,0.0,25.0,0.0,0.0,1.907893,0.0,0.0,0.0,0.0,0.0,0.0,0.111368,0.060643,0.009909,12.173937,12.17611,12.17821,1.907893,1.907893,1.907893


### Estados repetidos

En este caso se eliminan estados repetidos y estados que se encuentren en estado transitorio mientras el dron despega o se estabiliza antes de introducir la señal de control.

In [7]:
shape_b4 = dataset.shape
dataset = dataset.drop(["timestamps"], axis=1).drop_duplicates()
print(f'len (b4 drop) - len = {shape_b4[0]-dataset.shape[0]}')

len (b4 drop) - len = 495630


### División del dataset en estados y acciones

In [8]:
actions = dataset[rpm_list]
actions.describe()

,RPM0,RPM1,RPM2,RPM3
count,8.791987e+06,8.791987e+06,8.791987e+06,8.791987e+06
mean,1.439225e+04,1.439225e+04,1.439225e+04,1.439224e+04
std,1.754389e+03,1.754389e+03,1.754391e+03,1.754391e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.436491e+04,1.436492e+04,1.436492e+04,1.436491e+04
50%,1.447017e+04,1.447017e+04,1.447017e+04,1.447017e+04
75%,1.465628e+04,1.465628e+04,1.465628e+04,1.465628e+04
max,2.166645e+04,2.166645e+04,2.166645e+04,2.166645e+04


#### Normalización de acciones

In [9]:
def normalize_df(df):
    K = df.max().max()
    df_norm = actions/K
    return df_norm, {'K':K}

In [10]:
actions, K = normalize_df(actions)
print(K)
#actions_norm.describe()

{'K': 21666.4475}


#### Definimos los estados

In [11]:
states = dataset[states_list]
print(f'columns = {states.columns}')
print(f'shape = {states.shape}')
#states.head()

columns = Index(['vz', 'az', 'uvz', 'vz1', 'vz2', 'vz3', 'az1', 'az2', 'az3', 'uvz1',
       'uvz2', 'uvz3'],
      dtype='object')
shape = (8791987, 12)


In [12]:
del dataset
gc.collect()

80

### Dividimos el dataset

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(states, actions, test_size=0.1)
del states, actions
gc.collect()

20

## Keras Model

## Callbacks

#### Early Stopping

In [14]:
Early_Stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

### Sintonización de Hyperparámetros

In [16]:
from kerastuner import HyperModel
from kerastuner.tuners import Hyperband

#### Definición del modelo

In [17]:
class ANNHyperModel(HyperModel):
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape

    def build(self, hp):
        model = tf.keras.Sequential()
        model.add( tf.keras.Input(shape=(self.input_shape, )) )
        
        model.add(
            layers.Dense(
                units=hp.Int('units', self.input_shape, 64, 8, default=self.input_shape),
                activation='relu'
            )
        )
        model.add(
            layers.Dropout(rate=hp.Float(
                'dropout',
                min_value=0.0,
                max_value=0.3,
                default=0.2,
                step=0.05,
            ))
        )
        for i in range(hp.Int('num_layers', 1, 3)):
            model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=self.input_shape,
                                            max_value=64,
                                            step=8),
                               activation='relu'))
            
        model.add(layers.Dense(self.output_shape, activation=tf.nn.sigmoid))
        

        model.compile(
            optimizer=tf.keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='mean_squared_error',
            metrics=['mean_squared_error']
        )
        return model

#### Compilado del Modelo

In [30]:
HYPERBAND_MAX_EPOCHS = 40
HYPERBAND_ITERATIONS = 20
EXECUTION_PER_TRIAL = 2
N_EPOCH_SEARCH = 40
BATCH_SIZE = 50000
SEED = 10

In [31]:
hypermodel = ANNHyperModel(input_shape=X_train.shape[1], output_shape=Y_train.shape[1])
tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_mean_squared_error',
    seed=SEED,
    hyperband_iterations = HYPERBAND_ITERATIONS,
    executions_per_trial= EXECUTION_PER_TRIAL,
    directory=os.path.normpath('D:/'),
    project_name='Controller '+dataset_name
)
tuner.search_space_summary()

Search space summary
Default search space size: 5
units (Int)
{'default': 12, 'conditions': [], 'min_value': 12, 'max_value': 64, 'step': 8, 'sampling': None}
dropout (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.05, 'sampling': None}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 12, 'max_value': 64, 'step': 8, 'sampling': None}
learning_rate (Float)
{'default': 0.001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


#### Entrenamiento del Modelo

In [32]:
%%time
tuner.search(X_train, Y_train, epochs=N_EPOCH_SEARCH, validation_split=0.2, batch_size=BATCH_SIZE, callbacks=[Early_Stopping], verbose=1)

Trial 12 Complete [00h 00m 13s]
val_mean_squared_error: 0.00017446925630792975

Best val_mean_squared_error So Far: 9.66827356023714e-05
Total elapsed time: 00h 03m 37s

Search: Running Trial #13

Hyperparameter    |Value             |Best Value So Far 
units             |28                |44                
dropout           |0.25              |0                 
num_layers        |3                 |2                 
units_0           |20                |28                
learning_rate     |0.00034913        |0.0090513         
units_1           |36                |44                
units_2           |20                |60                
tuner/epochs      |2                 |5                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |1                 |0                 
tuner/round       |0                 |0                 

Epoch 1/2
 934/1126 [=======================>......] - ETA: 0s - loss: 0.0134 - mean_squared_error: 0.0134 ETA: 1s - l

KeyboardInterrupt: 

In [33]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 44 and the optimal learning rate for the optimizer
is 0.00905127409782462.



In [34]:
models = tuner.get_best_models(num_models=3)

Epoch 1/5
1126/1126 [==============================] - 6s 5ms/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 1.0741e-04 - val_mean_squared_error: 1.0741e-04
Epoch 2/5
1126/1126 [==============================] - 5s 5ms/step - loss: 1.0191e-04 - mean_squared_error: 1.0191e-04 - val_loss: 9.5755e-05 - val_mean_squared_error: 9.5755e-05
Epoch 3/5
1126/1126 [==============================] - 6s 5ms/step - loss: 9.4938e-05 - mean_squared_error: 9.4938e-05 - val_loss: 9.4080e-05 - val_mean_squared_error: 9.4080e-05
Epoch 4/5
1126/1126 [==============================] - 5s 5ms/step - loss: 9.3294e-05 - mean_squared_error: 9.3294e-05 - val_loss: 9.2617e-05 - val_mean_squared_error: 9.2617e-05
Epoch 5/5
1126/1126 [==============================] - 6s 5ms/step - loss: 8.9040e-05 - mean_squared_error: 8.9040e-05 - val_loss: 9.6163e-05 - val_mean_squared_error: 9.6163e-05
Best epoch: 1


In [ ]:
tuner.results_summary()

#### Evaluación del Modelo

In [35]:
eval_result = models[0].evaluate(X_test, Y_test)
print("[test_loss, test_mean_squared_error]:", eval_result)

54950/54950 [==============================] - 115s 2ms/step - loss: 9.3020e-05 - mean_squared_error: 9.3020e-050s - loss: 9.3069e-05 - me
[test_loss, test_mean_squared_error]: [9.30200912989676e-05, 9.30200912989676e-05]


#### Se guarda el Modelo

In [36]:
for i in models:    
    I = "Hp_Tunning_"+str(i)
    model.save(f'../Models/{dataset_name}_{I}.h5')

In [37]:
model = tf.keras.models.load_model(f'../Models/{dataset_name}_{I}.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 44)                572       
_________________________________________________________________
dropout_1 (Dropout)          (None, 44)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 28)                1260      
_________________________________________________________________
dense_7 (Dense)              (None, 44)                1276      
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 180       
Total params: 3,288
Trainable params: 3,288
Non-trainable params: 0
_________________________________________________________________


In [38]:
%%time
x_test = X_test.sample(n=3, random_state=1)
for index, sample in x_test.iterrows():
    print(model.predict([list(sample)])*K['K'])

[[14422.158 14432.703 14432.419 14439.1  ]]
[[13194.632  13218.592  13181.004  13178.7295]]
[[14610.773 14608.131 14613.357 14619.333]]
Wall time: 254 ms


In [39]:
%%time
x_test = [0]*len(states_list)
x_test[0] = 1
x_test[1] = -9.8
x_test[2] = 3
print(model.predict([list(x_test)])*K['K'])

[[21374.953 21337.012 21357.309 21373.518]]
Wall time: 52 ms
